# Classification with Cross Validation (CV) using sklearn

The first problem we have to attack is to define a metric we want to maximize (or minimize if it is a loss), and we will use it to select the best (or bests) models and hyperparameters. In order to understand the mechanism, we will code the validation loop by hand, and then we will use sklearn functions which do it automatically.

Although we are only interested in the metric on the validation set, it is interesting to compare it with the metric on the training set, to check the presence of overfitting.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import scipy
import sklearn
# plt.style.use('fivethirtyeight')
sns.set_style("whitegrid")
sns.set_context("notebook")
DATA_PATH = '../data/'

VAL_SPLITS = 4

In [2]:
from plot_utils import plot_confusion_matrix
from cv_utils import run_cv_f1

In [48]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier

For this part of the project, we will only work with the training set, that we will split again into train and validation to perform the hyperparameter tuning.

We will save the test set for the final part, when we have already tuned our hyperparameters.

In [3]:
df = pd.read_csv(os.path.join(DATA_PATH,'df_train.csv'))
df.drop(columns= df.columns[0:2],inplace=True)
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V24,V25,V26,V27,V28,Class,TimeScaled,TimeSin,TimeCos,AmountBC
0,-0.829392,1.118573,0.926038,1.163686,0.009824,0.527347,0.173370,0.723997,-0.638939,-0.162923,...,-0.298908,-0.060301,-0.217935,0.291312,0.120779,0,0.460069,-0.480989,0.876727,3.195062
1,-2.814527,1.613321,0.654307,0.581821,0.399491,0.730040,0.456233,-2.464347,0.654797,2.248682,...,-0.329526,-0.307374,-0.440007,-2.135657,0.011041,0,0.266395,-0.204567,-0.978853,3.125269
2,2.105028,-0.700400,-1.338043,-0.596395,-0.395217,-0.755050,-0.276951,-0.291562,-0.965418,1.107179,...,-0.278137,-0.040685,0.789267,-0.066054,-0.069956,0,0.762303,-0.153992,-0.988072,3.421235
3,2.205839,-1.023897,-1.270137,-0.950174,-0.868712,-0.975492,-0.475464,-0.280564,0.503713,0.448173,...,-0.041177,0.089158,1.105794,-0.066285,-0.079881,0,0.879740,-0.998227,0.059524,1.072145
4,2.027090,-0.778666,-1.552755,-0.558679,0.020939,-0.026071,-0.207810,-0.124288,-0.635953,0.817757,...,0.033477,-0.157992,-0.606327,-0.003931,-0.039868,0,0.821649,-0.783558,-0.621319,3.971490


## Manual cross validation (Exercise, built the CV loop)

We can do it manually, although it is better to define a function to help us iterate the CV over different algorithms and hyperparameters. It is more convenient to create a function, which is `run_cv_f1`

This is a typical CV loop, built from scratch

In [6]:
cv = StratifiedShuffleSplit(n_splits=VAL_SPLITS,test_size=0.15,random_state=0)
clf = LogisticRegression(solver='sag',random_state=0,n_jobs=-1)

# In case we want to select a subset of features
df_ = df
X = df_.drop(columns='Class').to_numpy()
y = df_['Class'].to_numpy()

# We create two eampty lists to save the metrics at each fold for train and validation. 
metrics = []
metrics_train = []
# Loop over the different validation folds
for i,(idx_t, idx_v) in enumerate(cv.split(X,y)):
    X_train = X[idx_t]
    y_train = y[idx_t]
    X_val = X[idx_v]
    y_val = y[idx_v]
    
    clf.fit(X_train,y_train)
    
    y_pred = clf.predict(X_val)
    metric = f1_score(y_val,y_pred)
    metrics.append(metric)
    
    y_t_pred = clf.predict(X_train)
    metric_train = f1_score(y_train,y_t_pred)
    metrics_train.append(metric_train)
    
    print('{}-fold / {} completed!'.format(i+1,VAL_SPLITS))
    
metric_mean = np.mean(metrics)
metric_std = np.std(metrics, ddof=1)
metric_t_mean = np.mean(metrics_train)
metric_t_std = np.std(metrics_train, ddof=1)
print('Metric value (Train): {:.2f} ± {:.2f}'.format(metric_t_mean,metric_t_std))
print('Metric value(Val): {:.2f} ± {:.2f}'.format(metric_mean,metric_std))

/Users/danky/anaconda3/envs/fraud_credit/lib/python3.6/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


1-fold / 4 completed!


/Users/danky/anaconda3/envs/fraud_credit/lib/python3.6/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


2-fold / 4 completed!


/Users/danky/anaconda3/envs/fraud_credit/lib/python3.6/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


3-fold / 4 completed!
4-fold / 4 completed!
Metric value (Train): 0.67 ± 0.01
Metric value(Val): 0.69 ± 0.04


/Users/danky/anaconda3/envs/fraud_credit/lib/python3.6/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


The same code can be casted in the form of a function, `run_cv_f1`, which can be found in `cv_utils.py`

In [5]:
cv = StratifiedShuffleSplit(n_splits=VAL_SPLITS,test_size=0.15,random_state=0)
clf = LogisticRegression(solver='sag',random_state=0,n_jobs=-1)

# In case we want to select a subset of features
df_ = df[['Class','V12','AmountBC','V16','V9']]
X = df_.drop(columns='Class').to_numpy()
y = df_['Class'].to_numpy()

run_cv_f1(clf,cv,X,y)

1-fold / 4 completed!
2-fold / 4 completed!
3-fold / 4 completed!
4-fold / 4 completed!
F1 value (Train): 0.61 ± 0.01
F1 value (Val): 0.63 ± 0.06


([0.6105263157894737,
  0.5869565217391305,
  0.608695652173913,
  0.7200000000000001],
 [0.6099815157116452,
  0.613970588235294,
  0.608534322820037,
  0.5906542056074766])

In [6]:
cv = StratifiedShuffleSplit(n_splits=VAL_SPLITS,test_size=0.15,random_state=0)
clf = RandomForestClassifier(n_estimators=100,n_jobs=-1,random_state=0)

# In case we want to select a subset of features
df_ = df[['Class','V12','AmountBC','V16','V9']]
X = df_.drop(columns='Class').to_numpy()
y = df_['Class'].to_numpy()

run_cv_f1(clf,cv,X,y)


1-fold / 4 completed!
2-fold / 4 completed!
3-fold / 4 completed!
4-fold / 4 completed!
F1 value (Train): 1.00 ± 0.00
F1 value (Val): 0.77 ± 0.05


([0.7747747747747747,
  0.6981132075471699,
  0.7884615384615384,
  0.8288288288288289],
 [1.0, 1.0, 0.9985358711566619, 1.0])

In [7]:
for name, value in zip(df_.drop(columns='Class').columns,100*clf.feature_importances_):
    print(name,value)

V12 45.168946351201825
AmountBC 8.546067680924292
V16 27.534791680607483
V9 18.750194287266396


In [8]:
cv = StratifiedShuffleSplit(n_splits=VAL_SPLITS,test_size=0.15,random_state=0)
clf = AdaBoostClassifier(random_state=0)

# In case we want to select a subset of features
df_ = df[['Class','V12','AmountBC','V16','V9']]
X = df_.drop(columns='Class').to_numpy()
y = df_['Class'].to_numpy()

run_cv_f1(clf,cv,X,y)


1-fold / 4 completed!
2-fold / 4 completed!
3-fold / 4 completed!
4-fold / 4 completed!
F1 value (Train): 0.68 ± 0.03
F1 value (Val): 0.66 ± 0.08


([0.6732673267326732,
  0.5625000000000001,
  0.6470588235294117,
  0.7547169811320754],
 [0.7177700348432056,
  0.6884681583476764,
  0.6598290598290599,
  0.6514886164623468])

In [10]:
for name, val in zip(df_.drop(columns='Class').columns,100*clf.feature_importances_):
    print(name,val)

V12 44.0
AmountBC 14.000000000000002
V16 18.0
V9 24.0


In [11]:
from sklearn.pipeline import Pipeline

In [40]:
scaler = StandardScaler()
clf_ = LogisticRegression(solver='lbfgs',random_state=0,class_weight='balanced',n_jobs=-1)
clf = Pipeline([('scaler', scaler), ('LogReg', clf_)])

# In case we want to select a subset of features
df_ = df[['Class','V4','V14','V16','V17','V18','TimeSin','AmountBC']]
X = df_.drop(columns='Class').to_numpy()
y = df_['Class'].to_numpy()

run_cv_f1(clf,cv,X,y)
print()

1-fold / 4 completed!
2-fold / 4 completed!
3-fold / 4 completed!
4-fold / 4 completed!
F1 value (Train): 0.08 ± 0.01
F1 value (Val): 0.08 ± 0.00



In [39]:
clf = LogisticRegression(solver='lbfgs',random_state=0,class_weight='balanced',n_jobs=-1)


# In case we want to select a subset of features
df_ = df[['Class','V4','V14','V16','V17','V18','TimeSin','AmountBC']]
X = df_.drop(columns='Class').to_numpy()
y = df_['Class'].to_numpy()

run_cv_f1(clf,cv,X,y)
print()

1-fold / 4 completed!
2-fold / 4 completed!
3-fold / 4 completed!
4-fold / 4 completed!
F1 value (Train): 0.08 ± 0.01
F1 value (Val): 0.08 ± 0.00



In [38]:
scaler = StandardScaler()
clf_ = LogisticRegression(solver='lbfgs',random_state=0,class_weight=None,n_jobs=-1)
clf = Pipeline([('scaler', scaler), ('LogReg', clf_)])

# In case we want to select a subset of features
df_ = df[['Class','V4','V14','V16','V17','V18','TimeSin','AmountBC']]
X = df_.drop(columns='Class').to_numpy()
y = df_['Class'].to_numpy()

run_cv_f1(clf,cv,X,y)
print()

1-fold / 4 completed!
2-fold / 4 completed!
3-fold / 4 completed!
4-fold / 4 completed!
F1 value (Train): 0.67 ± 0.01
F1 value (Val): 0.66 ± 0.05



In [37]:
clf = LogisticRegression(solver='lbfgs',random_state=0,class_weight=None,n_jobs=-1)

# In case we want to select a subset of features
df_ = df[['Class','V12','AmountBC','V16','V9']]
X = df_.drop(columns='Class').to_numpy()
y = df_['Class'].to_numpy()

run_cv_f1(clf,cv,X,y)
print()

1-fold / 4 completed!
2-fold / 4 completed!
3-fold / 4 completed!
4-fold / 4 completed!
F1 value (Train): 0.61 ± 0.01
F1 value (Val): 0.63 ± 0.06



In [41]:
scaler = StandardScaler()
clf_ = KNeighborsClassifier(n_neighbors=3,n_jobs=-1)
clf = Pipeline([('scaler', scaler), ('LogReg', clf_)])

# In case we want to select a subset of features
df_ = df[['Class','V4','V14','V16','V17','V18','TimeSin','AmountBC']]
X = df_.drop(columns='Class').to_numpy()
y = df_['Class'].to_numpy()

run_cv_f1(clf,cv,X,y)
print()

1-fold / 4 completed!
2-fold / 4 completed!
3-fold / 4 completed!
4-fold / 4 completed!
F1 value (Train): 0.87 ± 0.01
F1 value (Val): 0.81 ± 0.07



In [42]:
clf = KNeighborsClassifier(n_neighbors=3,n_jobs=-1)

# In case we want to select a subset of features
df_ = df[['Class','V4','V14','V16','V17','V18','TimeSin','AmountBC']]
X = df_.drop(columns='Class').to_numpy()
y = df_['Class'].to_numpy()

run_cv_f1(clf,cv,X,y)
print()

1-fold / 4 completed!
2-fold / 4 completed!
3-fold / 4 completed!
4-fold / 4 completed!
F1 value (Train): 0.87 ± 0.01
F1 value (Val): 0.80 ± 0.06



In [43]:
clf = RandomForestClassifier(n_estimators=100,n_jobs=-1,random_state=0)

# In case we want to select a subset of features
df_ = df[['Class','V4','V14','V16','V17','V18','TimeSin','AmountBC']]
X = df_.drop(columns='Class').to_numpy()
y = df_['Class'].to_numpy()

run_cv_f1(clf,cv,X,y)
print()

1-fold / 4 completed!
2-fold / 4 completed!
3-fold / 4 completed!
4-fold / 4 completed!
F1 value (Train): 1.00 ± 0.00
F1 value (Val): 0.80 ± 0.06



In [44]:
clf = AdaBoostClassifier()

# In case we want to select a subset of features
df_ = df[['Class','V4','V14','V16','V17','V18','TimeSin','AmountBC']]
X = df_.drop(columns='Class').to_numpy()
y = df_['Class'].to_numpy()

run_cv_f1(clf,cv,X,y)
print()

1-fold / 4 completed!
2-fold / 4 completed!
3-fold / 4 completed!
4-fold / 4 completed!
F1 value (Train): 0.74 ± 0.01
F1 value (Val): 0.73 ± 0.05



In [45]:
clf = ExtraTreesClassifier(n_estimators=50,n_jobs=-1,random_state=0)

# In case we want to select a subset of features
df_ = df[['Class','V4','V14','V16','V17','V18','TimeSin','AmountBC']]
X = df_.drop(columns='Class').to_numpy()
y = df_['Class'].to_numpy()

run_cv_f1(clf,cv,X,y)
print()

1-fold / 4 completed!
2-fold / 4 completed!
3-fold / 4 completed!
4-fold / 4 completed!
F1 value (Train): 1.00 ± 0.00
F1 value (Val): 0.81 ± 0.06



In [46]:
clf = ExtraTreesClassifier(n_estimators=50,n_jobs=-1,random_state=0)

# In case we want to select a subset of features
df_ = df
X = df_.drop(columns='Class').to_numpy()
y = df_['Class'].to_numpy()

run_cv_f1(clf,cv,X,y)
print()

1-fold / 4 completed!
2-fold / 4 completed!
3-fold / 4 completed!
4-fold / 4 completed!
F1 value (Train): 1.00 ± 0.00
F1 value (Val): 0.82 ± 0.04



In [49]:
scaler = StandardScaler()
clf_ = MLPClassifier(solver='lbfgs', alpha=1e-5,
                 hidden_layer_sizes=(16,8,2), random_state=0)
clf = Pipeline([('scaler', scaler), ('clf_', clf_)])

# In case we want to select a subset of features
df_ = df[['Class','V4','V14','V16','V17','V18','TimeSin','AmountBC']]
X = df_.drop(columns='Class').to_numpy()
y = df_['Class'].to_numpy()

run_cv_f1(clf,cv,X,y)
print()

1-fold / 4 completed!
2-fold / 4 completed!
3-fold / 4 completed!
4-fold / 4 completed!
F1 value (Train): 0.81 ± 0.01
F1 value (Val): 0.78 ± 0.05



In [50]:
scaler = StandardScaler()
clf_ = MLPClassifier(solver='lbfgs', alpha=1e-5,
                 hidden_layer_sizes=(16,8,2), random_state=0)
clf = Pipeline([('scaler', scaler), ('clf_', clf_)])

# In case we want to select a subset of features
df_ = df
X = df_.drop(columns='Class').to_numpy()
y = df_['Class'].to_numpy()

run_cv_f1(clf,cv,X,y)
print()

1-fold / 4 completed!
2-fold / 4 completed!
3-fold / 4 completed!
4-fold / 4 completed!
F1 value (Train): 0.97 ± 0.01
F1 value (Val): 0.79 ± 0.04



## Using sklearn function for CV (Preferred)

Even if it is easy to build the CV loop, and we can change the metric and the outputs in a personalized way, using a `for` loop makes the CV step non parallelizable (we could use the library `multiprocessing`) to solve this, but `sklearn` has implemented such an utility.

These utilities can be found in sklearn's [webpage](https://scikit-learn.org/stable/modules/classes.html#model-validation). Some of them are:
* [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) : Returning the validation score for some given metric.
* [cross_validate](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate) : Evaluate metric(s) by cross-validation and also record fit/score times. It can also returns the train metric.

In [51]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

In [53]:
val_split = StratifiedShuffleSplit(n_splits=VAL_SPLITS,test_size=0.15,random_state=0)
clf = ExtraTreesClassifier(n_estimators=50,n_jobs=-1,random_state=0)
df_ = df[['Class','V9','V14','V16']]
X = df_.drop(columns='Class').to_numpy()
y = df_['Class'].to_numpy()
metrics = cross_val_score(clf,X,y,cv=val_split,scoring='f1',n_jobs=-1)
print('F1 value (Val): {:.2f} ± {:.2f}'.format(
                np.mean(metrics),
                np.std(metrics, ddof=1)
            ))

F1 value (Val): 0.82 ± 0.06


In [58]:
val_split = StratifiedShuffleSplit(n_splits=VAL_SPLITS,test_size=0.15,random_state=0)
clf = ExtraTreesClassifier(n_estimators=50,n_jobs=-1,random_state=0)
df_ = df[['Class','V9','V14','V16']]
X = df_.drop(columns='Class').to_numpy()
y = df_['Class'].to_numpy()
scores = cross_validate(clf,X,y,cv=val_split,scoring='f1',n_jobs=-1, return_train_score=True)
print('F1 value (Train): {:.2f} ± {:.2f}'.format(
                np.mean(scores['train_score']),
                np.std(scores['train_score'], ddof=1)
            ))
print('F1 value (Val): {:.2f} ± {:.2f}'.format(
                np.mean(scores['test_score']),
                np.std(scores['test_score'], ddof=1)
            ))

F1 value (Train): 1.00 ± 0.00
F1 value (Val): 0.82 ± 0.06


In [59]:
from sklearn.calibration import CalibratedClassifierCV

clf_ = ExtraTreesClassifier(n_estimators=50,n_jobs=-1,random_state=0)
cv = StratifiedShuffleSplit(n_splits=2,test_size=0.2,random_state=0)
# Calibrated with isotonic calibration
clf = CalibratedClassifierCV(clf_, cv=cv, method='isotonic')
df_ = df
X = df_.drop(columns='Class').to_numpy()
y = df_['Class'].to_numpy()
metrics = cross_val_score(clf,X,y,cv=val_split,scoring='f1',n_jobs=-1)
print('F1 value (Val): {:.2f} ± {:.2f}'.format(
                np.mean(metrics),
                np.std(metrics, ddof=1)
            ))

F1 value (Val): 0.82 ± 0.04
